<a href="https://colab.research.google.com/github/gopaldewoolkar/AgenticAI/blob/main/sql_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langgraph langchain_openai langchain_community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [3]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("Chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [5]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [7]:
import sqlite3
import pandas as pd
from IPython.display import display
from langchain_community.utilities import SQLDatabase

#Connect to the SQLite database
conn = sqlite3.connect("Chinook.db")

# Write a query to get the data from the table
# query = "SELECT * FROM Customer"  # Customer table
query = "SELECT * FROM Invoice"   # Invoice table

# Load the data into a DataFrame
df = pd.read_sql_query(query, conn)

#Display the DataFrame
display(df)

# Close the connection
conn.close()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [8]:
!pip install langchain langchain_openai langchain_community langchain_experimental langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.3 MB/s eta 0:00:00


In [9]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("groq_api_key")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langsmith_api_key")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "SQL AGENT"

In [10]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 3.1 MB/s eta 0:00:00


In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0)

In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [13]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
import pprint

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

sql_tools = toolkit.get_tools()

pprint.pprint(sql_tools)

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f2defaa96f0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f2defaa96f0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f2defaa96f0>),
 QuerySQLCheckerTool(description='Use this tool to double check

## SQL Agents Prompt and Variables

In [14]:
from langchain import hub
import os

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
print(prompt_template.input_variables)

# Provide the required variables 'dialect' and 'top_k' to the prompt_template
formatted_prompt_template = prompt_template.format(dialect="sqlite", top_k=5)

['dialect', 'top_k']


## Parser Util function

In [17]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def parse_agent_messages(messages):
  for msg in messages:
    if isinstance(msg, HumanMessage):
      print(f"User : {msg.content}\n")
    elif isinstance(msg, AIMessage):
      if "tool_calls" in msg.additional_kwargs and msg.additional_kwargs["tool_calls"]:
        print("Agent is deciding to use tools....\n")
        for tool_call in msg.additional_kwargs["tool_calls"]:
          tool_name = tool_call['function']['name']
          arguments = tool_call['function']['arguments']
          print(f"Agent calls tool: {tool_name} with arguments {arguments}\n")
      else:
        print(f"Agent's Final Response:\n{msg.content}\n")
    elif isinstance(msg, ToolMessage):
      tool_name = msg.name
      print(f"Tool [{tool_name}] Response:\n{msg.content}")
    else:
      print(f"Unknown message type: {msg}\n")

## SQL Agent

In [15]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

sql_agent = create_react_agent(llm, sql_tools, state_modifier=formatted_prompt_template)

result = sql_agent.invoke(
    {"messages": [HumanMessage(content="List the total sales per country?")]}
)
pprint.pprint(result['messages'])

#print(result["messages"][-1].content)

#parse_agent_messages(result["messages"])

[HumanMessage(content='List the total sales per country?', additional_kwargs={}, response_metadata={}, id='f02a3cf5-d849-4831-a7a0-f4390b17ac50'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9svy', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}, {'id': 'call_6vxw', 'function': {'arguments': '{"table_names": "customers, orders, products"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_2pn2', 'function': {'arguments': '{"query": "SELECT country, SUM(total_amount) as total_sales FROM customers JOIN orders ON customers.customer_id = orders.customer_id GROUP BY country ORDER BY total_sales DESC LIMIT 5"}', 'name': 'sql_db_query_checker'}, 'type': 'function'}, {'id': 'call_e8bz', 'function': {'arguments': '{"query": "SELECT country, SUM(total_amount) as total_sales FROM customers JOIN orders ON customers.customer_id = orders.customer_id GROUP BY country ORDER BY total_sales DESC LIMIT 5"}', 'name': 'sql_db_query'}, 

In [16]:
sql_agent = create_react_agent(llm, sql_tools, state_modifier=formatted_prompt_template)

result = sql_agent.invoke(
    {"messages": [HumanMessage(content="List the total sales per country?")]}
)

print(result["messages"][-1].content)

The total sales per country are as follows:

- USA: $523.06
- Canada: $303.96
- France: $195.10
- Brazil: $190.10
- Germany: $156.48

These are the top 5 countries with the highest total sales.


In [18]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

sql_agent = create_react_agent(llm, sql_tools, state_modifier=formatted_prompt_template)

result = sql_agent.invoke(
    {"messages": [HumanMessage(content="List the total sales per country?")]}
)

parse_agent_messages(result["messages"])

User : List the total sales per country?

Agent is deciding to use tools....

Agent calls tool: sql_db_list_tables with arguments {}

Agent calls tool: sql_db_schema with arguments {"table_names": "sales, customers, products"}

Agent calls tool: sql_db_query_checker with arguments {"query": "SELECT country, SUM(sales_amount) as total_sales FROM sales GROUP BY country ORDER BY total_sales DESC LIMIT 5;"}

Agent calls tool: sql_db_query with arguments {"query": "SELECT country, SUM(sales_amount) as total_sales FROM sales GROUP BY country ORDER BY total_sales DESC LIMIT 5;"}

Tool [sql_db_list_tables] Response:
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Tool [sql_db_schema] Response:
Error: table_names {'sales', 'products', 'customers'} not found in database
Tool [sql_db_query_checker] Response:
```sql
SELECT "country", SUM("sales_amount") AS "total_sales" 
FROM "sales" 
GROUP BY "country" 
ORDER BY "total_sales" DESC 
LIMIT 5